<a href="https://colab.research.google.com/github/srilalithaveerubhotla/CMPE_258/blob/master/Project_Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
np.random.seed(42)
import requests
import io
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from sklearn import preprocessing 
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import dendrogram, cophenet,linkage

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df_31 = pd.read_csv(io.StringIO(uploaded['2019_nCoV_20200121_20200131.csv'].decode('utf-8')))
df_30 = pd.read_csv(io.StringIO(uploaded['2019_nCoV_20200121_20200130.csv'].decode('utf-8')))
df_28 = pd.read_csv(io.StringIO(uploaded['2019_nCoV_20200121_20200128.csv'].decode('utf-8')))
df_27 = pd.read_csv(io.StringIO(uploaded['2019_nCoV_20200121_20200127.csv'].decode('utf-8')))
df_26 = pd.read_csv(io.StringIO(uploaded['2019_nC0v_20200121_20200126_cleaned.csv'].decode('utf-8')))
df_sum = pd.read_csv(io.StringIO(uploaded['summary.csv'].decode('utf-8')))

In [0]:
df_sum.head(5)

In [0]:
df_sum=df_sum.fillna(value=0)

In [0]:
df_sum['Last Update'] =  pd.to_datetime(df_sum['Last Update'])

In [0]:
df_sum.groupby(by='Country').groups.keys()

In [0]:
df_sum.groupby(by='Last Update').groups.keys()

In [0]:
df_sum.groupby(by='Province/State').groups.keys()

In [0]:
country_count=pd.DataFrame(df_sum.groupby(['Country'])['Deaths'].count(),columns=['country','count'])

In [0]:
df_sum.head(3)

In [0]:
df_all = pd.concat([df_31,df_30,df_28], axis=0)

In [0]:
df_all=df_all.fillna(value=0)

In [0]:
df_all_1=df_all.drop_duplicates()

In [0]:
df_all_1.head(3)

In [0]:
df_all_1['last updated'] =  pd.to_datetime(df_all_1['Last Update'])

In [0]:
df_all_1['new_date'] = pd.to_datetime(df_all_1['Last Update']).dt.date

In [0]:
displays=['Confirmed','Recovered','Death','Suspected']

In [0]:
top5 =  df_all_1.groupby(['Country/Region']).sum().nlargest(5,['Confirmed'])[displays]
top5
print("Top 5 Countries were affected most")
print(top5)
top5=top5.reset_index()

In [0]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=90)
plt.title("Top most 5 countries were affected by Coronavirus")
sns.barplot(x=top5.index,y='Confirmed',data=top5)
plt.show()

In [0]:
df_all_1.head(50)

In [0]:
sns.pairplot(df_all_1)

In [0]:
count_all=df_all_1.groupby(by=['new_date'],axis=0).sum().reset_index()

In [0]:
count_all.head()

In [0]:
plt.figure(figsize=(15,6))
plt.xticks(rotation=90)
plt.title('Province/State has reported Deaths case')
sns.barplot(data=count_all, x='new_date', y='Death')

In [0]:
plt.figure(figsize=(15,6))
plt.xticks(rotation=90)
plt.title('Province/State has reported confirmed case')
sns.barplot(data=count_all, x='new_date', y='Confirmed')

In [0]:
plt.figure(figsize=(15,6))
plt.xticks(rotation=90)
plt.title('Province/State has reported recovered case')
sns.barplot(data=count_all, x='new_date', y='Recovered')

In [0]:
df_all_1=df_all_1.sort_values(by='Country/Region').reset_index()

In [0]:
df_all_1.head(30)

In [0]:
df = df_all_1[df_all_1['Province/State'] != 0]

In [0]:
df=df[df['Country/Region']!=0]

In [0]:
df=df.drop(['index','Country/Region'],axis=1)

In [0]:
df.head(3)

In [0]:
label_encoder = preprocessing.LabelEncoder() 

In [0]:
df['Province/State']= label_encoder.fit_transform(df['Province/State']) 

In [0]:
df.head(3)

In [0]:
df=df.drop(['Last Update','last updated','new_date'],axis=1)

In [0]:
kmeans = KMeans(n_clusters=8, init='k-means++', max_iter=300, n_init=10)
y_kmeans = kmeans.fit_predict(df)

In [0]:
plt.scatter(df.values[:, 2], df.values[:, 4], c=y_kmeans, s=100, alpha=0.5)

In [0]:
distortions = []
K = range(1,10)
for k in K:
    kmean = KMeans(n_clusters=k).fit(df)
    kmean.fit(df)
    distortions.append(sum(np.min(cdist(df, kmean.cluster_centers_, 'euclidean'), axis=1)) / df.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [0]:
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(df, method='ward'))
plt.axhline(y=3, color='r', linestyle='--')

In [0]:
dbscan = DBSCAN(eps=50,min_samples=8,algorithm='ball_tree')
db_clusters = dbscan.fit_predict(df)

In [0]:
plt.figure(figsize=(10, 10))
plt.scatter(df.values[:, 0], df.values[:, 4], c=db_clusters, s=100, alpha=0.5)
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")

In [0]:
df.describe()